In [1]:
import six.moves.urllib.request as urlreq

import dash
import dash_bio as dashbio
import dash_html_components as html
import dash_core_components as dcc

from dash_bio_utils import pdb_parser as parser
import json
import tempfile

import copy
import pandas as pd

try:
    import mutagenesis_visualization as mut
except ModuleNotFoundError:  # This step is only for when I run the notebooks locally
    import sys
    sys.path.append('../../../')
    import mutagenesis_visualization as mut

# local modules
try:
    import mutagenesis_visualization.main.scripts.code_kwargs as code_kwargs
    import mutagenesis_visualization.main.scripts.code_utils as code_utils
except ModuleNotFoundError:
    import import_notebook
    import code_kwargs
    import code_utils

In [2]:
hras_RBD = mut.hras_RBD()

In [21]:
pdb = '../../data/5p21.pdb'
dashboard_3d_protein(hras_RBD, pdb)

TypeError: The `dash_html_components.Div` component (version 1.1.1) with the ID "Div(Graph(id='right-column', figure=Figure({
    'data': [{'alignmentgroup': 'True',
              'bingroup': 'x',
              'histnorm': 'probability density',
              'hovertemplate': 'Score=%{x}<br>count=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa'},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'histogram',
              'x': array([-0.09092611,  0.08459133, -0.12058294, ...,  0.06633902,  0.023914  ,
                           0.129943  ]),
              'xaxis': 'x',
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'font': {'color': 'black', 'family': 'Arial, monospace', 'size': 12},
               'height': 240,
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
       

In [ ]:
def dashboard_3d_protein(
    self, pdb, mode='mean', chain='A', position_correction=0, **kwargs
):
    '''
    Docstring
    '''

    # update kwargs
    temp_kwargs = copy.deepcopy(code_kwargs.kwargs())
    temp_kwargs.update(kwargs)

    # Load data from pdb file
    model_data = _parse_pdb(pdb)

    # Calculate styles based on enrichment scores
    styles_data = _parse_styles_data(
        model_data, self.dataframe.copy(), temp_kwargs['gof'],
        temp_kwargs['lof'], mode, position_correction, chain
    )

    # Run dashhoard
    _run_dashboard_3d_protein(self, model_data, styles_data)


def _run_dashboard_3d_protein(self, model_data, styles_data):
    '''
    Create dashboard for main function.
    '''
    # Style sheet
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

    # Open app
    app = dash.Dash('__main__', external_stylesheets=external_stylesheets)

    # App layout
    app.layout = html.Div(
        id="app-container",
        children=[
            # first row
            html.Div([
        dcc.Graph( # For plotly graphs I need to use dcc
            id="heatmap",
            figure=self.histogram_plotly(
                return_plot_object=True, show=False,
                title='',
                x_label='Enrichment score',
                figsize=(3,2),

            )
        ),
                    ]),
            # second row
            html.Div([
                # Left column
                html.Div(
                    id="left-column",
                    className="six columns",
                    children=[
                        dashbio.Molecule3dViewer(
                            id='my-dashbio-molecule3d',
                            modelData=model_data,
                            styles=styles_data,
                            selectionType='Chain'
                        )
                    ]
                ),
                # Right column
                html.Div(
                    dcc.Graph( # For plotly graphs I need to use dcc
                        id="right-column",
                        className="six columns",
                        figure=self.histogram_plotly(
                            return_plot_object=True, show=False,
                            title='',
                            x_label='Enrichment score',
                            figsize=(3,2),

                        )
                    )
                ]),
            )
        ]
    )

    # Run server
    app.run_server(port=8070, )

In [4]:
def _parse_pdb(pdb):
    '''return the pdb in jason format'''
    # Parse pdb file
    modata = parser.create_data(pdb)

    # Put in jason format
    fmodel = files_data_style(modata)
    with open(fmodel) as fm:
        model_data = json.load(fm)

    return model_data


def _parse_styles_data(
    model_data, df, gof, lof, mode, position_correction, chain
):
    '''
    From a dataframe with enrichment scores, this function will return a jason file
    with the color of each atom.
    
    Returns
    -------
    styles_data : jason file
    '''

    # Create empty dict
    styles_data = {}

    # Calculate df with colors
    df_color = _add_color(
        hras_RBD.dataframe.copy(), 0.1, -0.3, chain, position_correction=0
    )

    # Iterate over parsed pdb
    for item in model_data['atoms']:
        #if item['chain'] != 'A':  # only color atoms from selected chain
        #break
        try:
            style_atom = {
                'color': _assign_color_jason(df_color, item['residue_index']),
                'visualization_type': 'cartoon'
            }
            styles_data[str(item['serial'])] = style_atom
        except IndexError:  # in case we runt out of index
            pass
    return styles_data


def _assign_color_jason(df, residue):
    '''
    Give a color to the atom based on the enrichment score of that residue.
    As an input takes the dataframe that _add_color returns.
    '''
    return df.loc[df['Position_Corrected'] == residue, 'Color'].iloc[0]


def _add_color(df, gof, lof, mode, position_correction):
    '''You input the dataframe. Removes stop codons. 
    Returns the positions that are going to be colored blue,red and white'''

    # Correct position
    df['Position_Corrected'] = df['Position'] + position_correction

    # Add dummy color column
    red = '#FD3216'
    blue = '#6A76FC'
    green = '#16FF32'
    df['Color'] = green

    # Select grouping
    if mode == 'mean':
        df_grouped = df.groupby(['Position'], as_index=False).mean()
    else:
        df_grouped = df.loc[df['Aminoacid'] == mode]

    # Color of mutations
    df_grouped.loc[df_grouped['Score'] < lof, 'Color'] = blue
    df_grouped.loc[df_grouped['Score'] > gof, 'Color'] = red

    return df_grouped


def files_data_style(content):
    '''
    Function to create the modelData and style files for molecule visualization
    '''
    fdat = tempfile.NamedTemporaryFile(suffix=".js", delete=False, mode='w+')
    fdat.write(content)
    dataFile = fdat.name
    fdat.close()
    return dataFile

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df_bar = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges",
              "Bananas"], "Amount": [4, 1, 2, 2, 4, 5], "City":
    ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df_bar, x="Fruit", y="Amount", color="City", barmode="group")

app.layout = html.Div(
    children=[
        # All elements from the top of the page
        html.Div([
            html.H1(children='Hello Dash'),
            html.Div(
                children='''
            Dash: A web application framework for Python.
        '''
            ),
            dcc.Graph(id='graph1', figure=fig),
        ]),
        # New Div for all elements in the new 'row' of the page
        html.Div([
            html.H1(children='Hello Dash'),
            html.Div(
                children='''
            Dash: A web application framework for Python.
        '''
            ),
            dcc.Graph(id='graph2', figure=fig),
        ]),
    ]
)

if __name__ == '__main__':
    app.run_server(port=8069)

Dash is running on http://127.0.0.1:8069/

Dash is running on http://127.0.0.1:8069/

Dash is running on http://127.0.0.1:8069/

Dash is running on http://127.0.0.1:8069/

Dash is running on http://127.0.0.1:8069/

Dash is running on http://127.0.0.1:8069/

Dash is running on http://127.0.0.1:8069/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
